In [19]:
import numpy as np


init_layers = [5, 4, 7, 4]

N_layers = len(init_layers)
l1 = np.random.randn(5)

print(l1)
print(type(l1))
layers = np.empty(N_layers, dtype='object')

for L, N_w in enumerate(init_layers):
    layers[L] = np.random.randn(N_w)
    
print(layers)

[-0.61641863  0.06032877 -0.71725196  0.21793259 -0.71646218]
<class 'numpy.ndarray'>
[array([ 0.53207798,  0.86320372,  0.42559635, -0.37871013,  0.04826926])
 array([ 0.50150889, -1.59830481, -0.58940219, -1.10671825])
 array([-1.08538395, -0.5978912 ,  1.23755473,  1.74329505, -0.82194983,
        2.677398  , -0.17277928])
 array([-0.36492219, -0.92060045,  0.1058421 , -0.42380144])]
